In [1]:
import os
import glob
import pandas as pd

In [2]:
os.chdir(r"C:\Users\polin\OneDrive\Документы\stocks")

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (0,9) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
data = pd.read_csv('combined_csv.csv', low_memory=False) 

In [4]:
data

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,<TICKER>;<PER>;<DATE>;<TIME>;<OPEN>;<HIGH>;<LOW>;<CLOSE>;<VOL>
0,AFKS,1.0,20200103.0,100100.0,15.252,15.299,15.252,15.295,65700.0,NaN
1,AFKS,1.0,20200103.0,100200.0,15.300,15.324,15.252,15.281,113500.0,NaN
2,AFKS,1.0,20200103.0,100300.0,15.281,15.335,15.281,15.296,42600.0,NaN
3,AFKS,1.0,20200103.0,100400.0,15.296,15.316,15.282,15.302,13100.0,NaN
4,AFKS,1.0,20200103.0,100500.0,15.309,15.322,15.290,15.293,75400.0,NaN
...,...,...,...,...,...,...,...,...,...,...
3538369,TATN,1.0,20200903.0,160400.0,532.000,532.000,531.700,532.000,5122.0,NaN
3538370,TATN,1.0,20200903.0,160500.0,531.900,532.000,531.700,531.900,10749.0,NaN
3538371,TATN,1.0,20200903.0,160600.0,531.900,532.400,531.900,532.400,4755.0,NaN
3538372,TATN,1.0,20200903.0,160700.0,532.400,532.700,532.200,532.700,3549.0,NaN


In [6]:
def calculate_adjusted_prices(df, column):
    """ Vectorized approach for calculating the adjusted prices for the
    specified column in the provided DataFrame. This creates a new column
    called 'adj_<column name>' with the adjusted prices. This function requires
    that the DataFrame have columns with dividend and split_ratio values.

    :param df: DataFrame with raw prices along with dividend and split_ratio
        values
    :param column: String of which price column should have adjusted prices
        created for it
    :return: DataFrame with the addition of the adjusted price column
    """
    adj_column = 'adj_' + column

    # Reverse the DataFrame order, sorting by date in descending order
    df.sort_index(ascending=False, inplace=True)

    price_col = df[column].values
    split_col = df['split_ratio'].values
    dividend_col = df['dividend'].values
    adj_price_col = np.zeros(len(df.index))
    adj_price_col[0] = price_col[0]

    for i in range(1, len(price_col)):
        print('adj', adj_price_col[i - 1])
        print('price', price_col[i])
        print('split', split_col[i - 1])
        print('div', dividend_col[i-1])
        print('bbb', (((price_col[i] * split_col[i - 1]) - price_col[i - 1] - dividend_col[i - 1]) / price_col[i - 1]))
        print('aaa', adj_price_col[i-1] * (((price_col[i] * split_col[i - 1]) - price_col[i - 1] -dividend_col[i - 1]) / price_col[i - 1]))
        adj_price_col[i] = adj_price_col[i-1] + adj_price_col[i-1] * (((price_col[i] * split_col[i - 1]) - price_col[i - 1] -dividend_col[i - 1]) / price_col[i - 1])
        #adj_price_col[i] = round((adj_price_col[i - 1] + adj_price_col[i - 1] *
                   #(((price_col[i] * split_col[i - 1]) -
                     #price_col[i - 1] -
                     #dividend_col[i - 1]) / price_col[i - 1])), 4)
        #print(adj_price_col[i])
        if i == 5:
            break

    df[adj_column] = adj_price_col

    # Change the DataFrame order back to dates ascending
    df.sort_index(ascending=True, inplace=True)

    return df

In [3]:
data = pd.read_csv('CHMF_200101_201231.csv') 

In [4]:
data.columns = ['TICKER','PER','DATE','TIME','OPEN','HIGH','LOW','CLOSE','VOL','dividend','split_ratio']

In [5]:
data.head()

,TICKER,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,dividend,split_ratio
0,CHMF,1,20200803,100100,913.0,915.0,912.4,914.0,4103,0.0,1
1,CHMF,1,20200803,100200,914.6,914.6,910.6,912.4,2388,0.0,1
2,CHMF,1,20200803,100300,912.0,913.4,912.0,912.8,1832,0.0,1
3,CHMF,1,20200803,100400,912.8,913.0,912.0,912.4,588,0.0,1
4,CHMF,1,20200803,100500,912.0,912.4,911.0,911.2,1866,0.0,1


In [103]:
test = data.sort_index(ascending = False)

In [104]:
test.head(10)

,TICKER,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,dividend,split_ratio
158083,AFKS,1,20201230,235000,28.714,28.730,28.707,28.711,13500,0.0,0
158082,AFKS,1,20201230,234900,28.714,28.715,28.714,28.714,4900,0.0,0
158081,AFKS,1,20201230,234800,28.710,28.711,28.710,28.710,4700,0.0,0
158080,AFKS,1,20201230,234700,28.710,28.710,28.707,28.710,6800,0.0,0
158079,AFKS,1,20201230,234600,28.721,28.738,28.707,28.707,54400,0.0,0
158078,AFKS,1,20201230,234500,28.717,28.738,28.715,28.738,4600,0.0,0
158077,AFKS,1,20201230,234400,28.717,28.721,28.710,28.715,45900,0.0,0
158076,AFKS,1,20201230,234300,28.726,28.726,28.718,28.718,55100,0.0,0
158075,AFKS,1,20201230,234200,28.734,28.734,28.734,28.734,700,0.0,0
158074,AFKS,1,20201230,234100,28.737,28.737,28.725,28.735,5600,0.0,0


In [9]:
import numpy as np

In [29]:
data = calculate_adjusted_prices(data,'HIGH')
data = calculate_adjusted_prices(data,'LOW')
data = calculate_adjusted_prices(data,'CLOSE')

In [35]:
data['dividend'].sum()

0.26899999999999996

In [8]:
data.drop('split_ratio',axis=1)

,TICKER,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,dividend
0,AFKS,1,20200103,100100,15.252,15.299,15.252,15.295,65700,0.0
1,AFKS,1,20200103,100200,15.300,15.324,15.252,15.281,113500,0.0
2,AFKS,1,20200103,100300,15.281,15.335,15.281,15.296,42600,0.0
3,AFKS,1,20200103,100400,15.296,15.316,15.282,15.302,13100,0.0
4,AFKS,1,20200103,100500,15.309,15.322,15.290,15.293,75400,0.0
...,...,...,...,...,...,...,...,...,...,...
158079,AFKS,1,20201230,234600,28.721,28.738,28.707,28.707,54400,0.0
158080,AFKS,1,20201230,234700,28.710,28.710,28.707,28.710,6800,0.0
158081,AFKS,1,20201230,234800,28.710,28.711,28.710,28.710,4700,0.0
158082,AFKS,1,20201230,234900,28.714,28.715,28.714,28.714,4900,0.0


In [7]:
def calculate_adjusted_prices(df, column):
    """ Vectorized approach for calculating the adjusted prices for the
    specified column in the provided DataFrame. This creates a new column
    called 'adj_<column name>' with the adjusted prices. This function requires
    that the DataFrame have columns with dividend and split_ratio values.

    :param df: DataFrame with raw prices along with dividend and split_ratio
        values
    :param column: String of which price column should have adjusted prices
        created for it
    :return: DataFrame with the addition of the adjusted price column
    """
    adj_column = 'adj_' + column

    # Reverse the DataFrame order, sorting by date in descending order
    df.sort_index(ascending=False, inplace=True)

    price_col = df[column].values
    split_col = df['split_ratio'].values
    dividend_col = df['dividend'].values
    adj_price_col = np.zeros(len(df.index))
    adj_price_col[0] = price_col[0]
    count = 0
    for i in range(1, len(price_col)):
        adj_price_col[i] = round(adj_price_col[i - 1] + adj_price_col[i - 1] * ((price_col[i] - price_col[i - 1] - dividend_col[i - 1]) / price_col[i - 1]), 4)

    df[adj_column] = adj_price_col
        

    # Change the DataFrame order back to dates ascending
    df.sort_index(ascending=True, inplace=True)

    return df

In [10]:
data = calculate_adjusted_prices(data,'OPEN')

In [11]:
data.head(10)

,TICKER,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,dividend,split_ratio,adj_OPEN
0,CHMF,1,20200803,100100,913.0,915.0,912.4,914.0,4103,0.0,1,735.6557
1,CHMF,1,20200803,100200,914.6,914.6,910.6,912.4,2388,0.0,1,736.9449
2,CHMF,1,20200803,100300,912.0,913.4,912.0,912.8,1832,0.0,1,734.8499
3,CHMF,1,20200803,100400,912.8,913.0,912.0,912.4,588,0.0,1,735.4945
4,CHMF,1,20200803,100500,912.0,912.4,911.0,911.2,1866,0.0,1,734.8499
5,CHMF,1,20200803,100600,911.0,911.0,910.4,910.8,2636,0.0,1,734.0441
6,CHMF,1,20200803,100700,910.8,911.8,910.4,911.8,3029,0.0,1,733.8829
7,CHMF,1,20200803,100800,911.4,912.0,911.0,911.0,1353,0.0,1,734.3664
8,CHMF,1,20200803,100900,911.4,911.4,910.6,910.6,432,0.0,1,734.3664
9,CHMF,1,20200803,101000,911.0,911.0,910.6,911.0,942,0.0,1,734.0441


In [12]:
data = calculate_adjusted_prices(data,'HIGH')
data = calculate_adjusted_prices(data,'LOW')
data = calculate_adjusted_prices(data,'CLOSE')

In [19]:
data

,TICKER,DATE,TIME,VOL,adj_OPEN,adj_HIGH,adj_LOW,adj_CLOSE
0,CHMF,20200803,100100,4103,735.6557,737.4270,735.1493,736.6045
1,CHMF,20200803,100200,2388,736.9449,737.1046,733.6990,735.3150
2,CHMF,20200803,100300,1832,734.8499,736.1375,734.8270,735.6374
3,CHMF,20200803,100400,588,735.4945,735.8151,734.8270,735.3150
4,CHMF,20200803,100500,1866,734.8499,735.3315,734.0213,734.3479
...,...,...,...,...,...,...,...,...
84560,CHMF,20201230,234600,211,1206.2076,1206.5474,1205.4076,1206.5300
84561,CHMF,20201230,234700,130,1206.3901,1206.9124,1205.5901,1206.7125
84562,CHMF,20201230,234800,955,1206.5726,1207.2774,1206.5025,1206.7125
84563,CHMF,20201230,234900,1451,1207.1200,1208.9200,1206.3200,1208.7200


In [14]:
data.drop(columns = ['OPEN','HIGH','LOW','CLOSE','dividend','split_ratio'], inplace = True)

In [15]:
data.head()

,TICKER,PER,DATE,TIME,VOL,adj_OPEN,adj_HIGH,adj_LOW,adj_CLOSE
0,CHMF,1,20200803,100100,4103,735.6557,737.4270,735.1493,736.6045
1,CHMF,1,20200803,100200,2388,736.9449,737.1046,733.6990,735.3150
2,CHMF,1,20200803,100300,1832,734.8499,736.1375,734.8270,735.6374
3,CHMF,1,20200803,100400,588,735.4945,735.8151,734.8270,735.3150
4,CHMF,1,20200803,100500,1866,734.8499,735.3315,734.0213,734.3479


In [16]:
data.drop(columns = ['PER'], inplace = True)

In [17]:
data.to_csv('CHMF.csv', index=False)